In [49]:
import pandas as pd
import json
import re
import string
import time
import csv

# shapely
from shapely.geometry import asShape
from shapely.ops import unary_union
from shapely.geometry import Polygon
import shapely.wkt
from shapely import wkb 
from shapely import wkt
# read both JSON and green text files 
# Convert to list of polygons
# For each polygon suburb calculate intersection area with green polygon area
# Add to the green area

start_time = time.time()
suburbsInput ='melb_inner_2016.json'
greenInputfilenames = ['part-00000', 'part-00001','part-00002','part-00003','part-00004','part-00005']
# suburbsInput ='two_suburbs.json'
# greenInputfilenames = ['two_lines.txt']
output_filename = "finalsa1_with_green_areas.csv"
# suburbsInput ='melb_inner_2016.json'
# greenInputfilenames = ['part-00000.txt', 'part-00001.txt','part-00002.txt','part-00003.txt','part-00004.txt','part-00005.txt']



# to replace every 2nd occurence with , so that it can be converted to polygon
def polystr_to_polygon(s, sub, repl):
    find = s.find(sub)
    # loop util we find no match
    i = 1
    while find != -1:
        if  (i - 1)/2  == (i - 1)//2 and i != 1 :
            s = s[:find]+repl+s[find + len(sub):]
        find = s.find(sub, find + len(sub) + 1)
        i += 1
    poly = shapely.wkt.loads(s)
    return poly

def read_melbourn_suburb_file(suburbsInput):
    jsonpdf = pd.read_csv(suburbsInput,sep= "}{",header=None)
    jsonpdf.columns = ['jsonstring']
    return jsonpdf

def return_polygons_list(jsonpdf):
    sa1 = list()
    for index, row in jsonpdf.iterrows():
        jsonline =''
        salStartPos =''
        sa1EndPos = ''
        coordStartPos = ''
        coordEndPos = ''
        jsonline = row['jsonstring']  
        sa1StartPos = re.search("\"sa1_7dig16\"\:",jsonline)
        sa1EndPos = sa1StartPos.start() + 14
        sa1.append(jsonline[sa1EndPos:sa1EndPos+7])
        coordStartPos = re.search("\"coordinates\"\:",jsonline)
        coordEndPos = coordStartPos.start() + 14
        coords = jsonline[coordEndPos:]
        modcds = []
        modcds = coords.replace("[[[[", "(").replace("]]]]", ")").replace("]", ")").replace("[", "(").replace("}}","")
        l = eval( "[%s]"%modcds)
        poly = Polygon(l)
        sa1.append(poly)
        sa1.append(round(poly.area * 10000,3))
    return sa1   

def read_green_areas(inputfile):
    greenjsonpdf = pd.read_csv(inputfile,sep= "\"",header=None)
    greenjsonpdf.columns = ['area','greenpolygon','field1']
    return greenjsonpdf


def main():
    
    sub = " "
    repl= ","
    fullGreenPolygons = list()

    # read suburbs file
    jsonpdf = read_melbourn_suburb_file(suburbsInput)

    # get polygon list of suburbs
    sa1 = return_polygons_list(jsonpdf)

    # iterate through mulitple input green area files
    for inputfilename in greenInputfilenames:
        fullgreenpdf = read_green_areas(inputfilename)
    # iterate through pdf to create polygon list        
        for index, row in fullgreenpdf.iterrows():
            polyline =''
            polyline = row['greenpolygon'].replace("\"","")
            greenpoly = polystr_to_polygon(polyline, " ", ",") 
            fullGreenPolygons.append(greenpoly)

    # print(len(fullGreenPolygons))
    # print(type(fullGreenPolygons))

    # Now iterate through SA1 list of polygons of suburbs with green areas of polygons
    # find the insterect area and append to total green area of the suburb(sa1)

    sa1index=0
    sa1polyindex= 1
    sa1areaindex= 2
    colindex =0
    finalsa1 = list()

    for sa1_row in sa1:
        if sa1index == colindex:
            finalsa1.append(sa1[sa1index])  
            sa1index += 3
        if sa1polyindex == colindex:
            intersectarea = 0  
            for greenpoly in fullGreenPolygons:  
                instersectpoly =sa1[sa1polyindex].intersection(greenpoly)
                if instersectpoly.area > 0:
    #                 print('test')
#                     print(instersectpoly.area)
                    intersectarea = round(intersectarea + (instersectpoly.area* 10000),10)
            sa1polyindex += 3
            finalsa1.append(sa1[sa1areaindex])
        if sa1areaindex == colindex:
            finalsa1.append(intersectarea)  
            sa1areaindex += 3    
        colindex +=1   
        
# write to outputfile
    outF = open(output_filename, "w")
    outF.write('SA1,Area,Green Area\n')
    i = 0
    filestring = ''
    for finalsal_row in finalsa1:
        if i ==0:
            filestring = str(finalsal_row)
        if i > 0:
            filestring = filestring + ',' + str(finalsal_row)
        i +=1
        if i == 3:
            filestring += '\n'
#             print(filestring)
            outF.write(filestring)
            i=0
            filestring = ''
    outF.close()       
    calc_time = time.time() - start_time
    print('Calculation time:', round(calc_time / 60, 2), 'min')
if __name__ == "__main__":
    main()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:45: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


1.378627430639374e-11
1.48437304234789e-11
1.5235190676175918e-10
2.3194806536674774e-10
1.6194289824736455e-10
1.115123073010408e-10
1.7033640208820385e-10
4.5225056673594667e-10
1.2227645491853516e-10
6.649526236411338e-11
2.134475091897254e-10
8.791442696859233e-11
2.035370595647572e-10
2.774617166537345e-10
4.314144715596988e-10
9.711099577790948e-11
2.0339447189673933e-10
1.0498877127559749e-10
2.678505539007498e-10
2.2092435193641525e-10
3.0756523470014866e-10
2.7368524375031376e-10
3.7765455916849724e-10
3.949877145744053e-10
1.708179657161933e-10
1.2590751584398618e-09
1.8341107682831007e-09
1.956883282037172e-10
4.3557283063197707e-10
6.983542927462116e-10
2.8830496468775986e-10
2.794557087340922e-10
1.7854610915681663e-10
2.8542600981104e-10
3.388726940597999e-10
1.8161119537928925e-10
5.75804372431795e-10
1.9889313386311514e-10
2.765642821473259e-10
2.1376014472119704e-10
2.4792635908079054e-10
1.6374796530528996e-10
1.8223296379306718e-10
1.0069335503128905e-10
1.7718252407

7.53619808619198e-11
2.5481059249411075e-11
7.198386915548651e-11
1.1654569865828868e-09
2.1653906404064276e-10
1.0965385836482243e-10
8.364934293518308e-11
9.91785999701126e-11
9.615840802639234e-11
4.7728313715166245e-11
8.149249533022011e-11
7.466584543661816e-11
6.855164068312494e-11
1.846670004128554e-10
6.879655770282712e-11
1.2994492288136927e-10
2.9945731128309195e-11
1.2038306506704998e-10
4.351135871706129e-11
5.912245726907962e-10
3.685378757758894e-10
7.831924412888904e-10
5.7999658809521705e-11
4.63544814998144e-12
2.5536315938155636e-12
8.708816046155184e-12
7.735710150667694e-11
9.19287457732291e-11
6.147485694499962e-11
8.965891604223083e-11
8.441782428387646e-11
6.566207234736048e-11
5.543902011959455e-11
7.192052986705347e-11
5.942576587532298e-11
5.941574652294784e-11
1.379599732424336e-09
4.383558568467433e-10
5.492255027727259e-10
3.631569500188871e-10
2.2673070504758467e-10
1.2161785376768384e-08
3.2150417054669697e-09
3.1880353122678576e-09
8.704899928909323e-09


2.702405875756327e-09
1.054556184168566e-09
5.534736534231318e-10
5.3465154037938204e-11
8.801606920736073e-11
8.29068917396723e-11
8.933604104653988e-11
9.23519066269932e-11
7.888672662323037e-11
9.468076520867788e-11
2.7641921407606266e-10
2.186821090704667e-10
2.832217179308319e-10
1.9961308845218156e-10
9.724361437153432e-11
1.2918362104011648e-10
7.558163726812692e-11
7.792277855371762e-11
1.113838423649257e-10
9.821488052367881e-11
6.780814883149639e-11
9.883350957704371e-11
2.949535086041077e-11
5.821187442040187e-11
2.2118257592486572e-11
1.3903890001638624e-11
5.559315525590336e-11
5.559315525590336e-11
3.370629370659761e-11
1.3050298931869266e-10
7.978926921072917e-11
1.5790311693262697e-10
2.6438524557162215e-10
1.9123940893636346e-10
6.232107852538241e-10
3.032579740553626e-10
9.474678662504849e-10
2.1999511369973962e-10
4.5704805562966354e-10
2.2198501307740245e-10
8.907464015447642e-10
2.0023318681517226e-10
1.4653322014692204e-09
9.007571064493856e-10
2.0770825463928222e

1.5380854608451722e-10
1.8790785733505833e-09
3.5674684877556296e-10
2.115787308264203e-10
2.5459376100260473e-10
2.104062388740923e-10
1.400047431076202e-10
1.607827733056031e-10
1.877370204189488e-10
1.1848607796420272e-10
1.343729251133952e-10
8.921834593784006e-11
1.566072996293963e-10
1.112285911583776e-10
1.1990365505542313e-10
1.1790134927804986e-10
7.310660932928653e-11
6.737286860260907e-11
1.0288230031204027e-10
1.2068050812574727e-10
1.2791861120195404e-10
7.355683535349741e-11
1.1213710671383613e-10
7.356182081477158e-11
7.318513853329838e-11
1.3071304128866238e-10
1.0247180269205553e-10
1.2396246353502723e-10
1.5341565692926713e-10
1.1211817065245325e-10
1.2164679995531793e-10
1.5601840816665007e-10
7.237141675335746e-11
1.5667341612460323e-10
1.6543662728985698e-09
3.2357603189184507e-10
4.725556386180091e-10
4.229410971102333e-10
4.610963481107728e-10
1.233946789900101e-10
6.396634985396638e-08
1.3030664914765892e-09
1.3207261525216036e-08
5.053878796005271e-08
2.0687615

...

3.8859989139255645e-10
3.1799438448220727e-10
2.3182791200018495e-10
2.0772928569087677e-10
6.153385621147891e-09
2.0501880790991302e-10
2.0107063393832548e-07
1.7302685990828326e-08
5.61662200612886e-10
3.622767310934106e-10
5.396840813288196e-10
3.1403507855349384e-10
4.79158117112793e-10
3.7545270289851186e-10
2.8216648215846805e-10
2.2870198580649549e-10
2.0144664497618693e-10
1.4265737533332306e-08
1.737665293752759e-10
1.1840093787036473e-10
6.925942230719097e-10
1.1433698529276668e-07
2.5299887363150997e-10
1.2740965730664145e-08
5.79129116829161e-10
2.405811696853938e-10
8.865846464655731e-09
3.3507475333682227e-10
4.138026983967183e-10
3.675402711814801e-10
3.461750858408284e-08
4.175613649893188e-10
2.106267434876864e-10
5.267604816329739e-09
4.113541871114262e-10
3.823954608681533e-10
3.6656387504942758e-09
3.5062080495601474e-09
5.336188054832756e-10
1.401675386770633e-09
3.335002984549456e-08
1.9234216043289086e-08
2.965790616054981e-10
1.21873014292647e-08
2.5698307267177

1.5694266452135186e-09
2.2705576858404076e-09
6.51305733353435e-10
3.100717119800181e-09
4.585613538968044e-10
4.95625436147079e-09
3.937021892861044e-09
5.324406868139077e-10
6.0704280782141305e-09
2.6783471074037267e-09
3.806456378643247e-09
6.272490048871812e-10
3.30860106941493e-08
2.6275846843711535e-09
2.7357357840033174e-07
1.6778622677080869e-09
4.4148144698772797e-10
1.0281634494721307e-08
2.1087119146585514e-08
9.67457911784349e-09
1.840364078360521e-09
7.198833697657525e-09
1.468664218850908e-09
3.7808043889438765e-10
4.0988702386140375e-09
1.8805146202328145e-10
3.455388104628367e-10
3.7125472867889805e-09
2.4086001010077437e-08
3.566222450569078e-10
6.06984487098004e-10
1.087644640890425e-09
1.5123201892489635e-09
3.3409963995524085e-09
1.7904395826347949e-09
1.1208282380837498e-08
5.1581083363127354e-09
2.8387591565027087e-09
3.6657471871735426e-09
7.667824742154234e-10
3.308786966020176e-09
7.603092548559153e-09
2.8112240628855575e-08
1.7863502472366694e-09
4.54093149332

2.1913308828535603e-10
6.0705271671372046e-09
2.1069130396824968e-10
1.995261457156767e-10
4.396357063824717e-08
1.5181562312255812e-10
2.6378668426363094e-08
9.723053544315349e-09
2.0926152715761992e-08
3.375053461193657e-08
9.865084222889513e-09
8.707995736101738e-09
4.113988345622588e-11
1.1690394334763522e-08
1.097112376007061e-08
4.7872870126249295e-11
6.358277183333158e-11
5.7305211615827567e-11
2.0018124202713597e-08
1.2130041249141991e-08
8.05923968284465e-09
4.476123968829552e-09
4.143828663331805e-09
3.675911601812803e-08
1.000359738407841e-10
1.4501358178381392e-08
4.402635624372524e-08
1.5403432562661657e-08
1.1673191438655174e-08
2.67936076798784e-08
1.9199229389977217e-08
9.32706071003931e-10
1.8690580895617798e-08
5.032466983760925e-09
2.2088483446371928e-08
8.161320005748855e-10
1.917864893846361e-08
5.3209288293899875e-09
5.307960923750766e-09
2.8717040299097342e-09
5.738918945224076e-09
5.135556366924979e-10
4.2763222145990406e-10
4.131070429797495e-10
2.6668390716371

5.3921729290796636e-09
8.292685634990532e-09
1.521721992763296e-08
1.0641191456741557e-08
4.759791127002922e-09
2.047456229997641e-08
7.111182068415859e-09
7.0328181170796045e-09
3.96445233161741e-09
6.05215083188797e-09
2.065229965642811e-08
9.38026244278754e-09
4.102625107538992e-09
3.1592812652007733e-09
2.1755033980773185e-08
2.304151270159413e-08
7.019974042624528e-09
6.523129052963227e-09
3.6293539786443747e-09
4.308667181162675e-09
2.9028826184321233e-09
3.835328726417852e-09
7.654973214947325e-09
3.5197167269269327e-09
3.5517722246332073e-09
4.711255664847645e-09
7.317764478120032e-09
2.5824296593007732e-08
1.9703719308183884e-08
2.8646556159833216e-10
2.0706884363039375e-10
4.5859107948252924e-11
3.1128636884811075e-10
2.1971107819926582e-08
1.1362906730287323e-08
8.20947025416626e-09
6.60318176269101e-09
9.193126539252017e-09
2.3275737432361896e-09
6.584762259180416e-09
3.0952468550883737e-09
2.0097371343980878e-08
1.6163370770883497e-09
1.593658660113288e-08
8.07790508031192

6.450209997021877e-09
1.626462133704431e-08
1.4409530078429967e-08
1.8877401530708738e-08
1.5356746275721716e-08
9.335139928964927e-10
1.5390133040147592e-10
3.462089778135538e-11
1.8737226062545306e-10
3.142088239817231e-10
5.386454375054445e-10
2.8059691768951376e-10
2.752611956107296e-11
3.707613979045149e-10
1.863956342435732e-10
1.409952359838419e-10
5.820209665452533e-10
6.000641010829476e-10
3.531122884942724e-10
1.2981910144835253e-09
2.6979458427193544e-10
2.059432238592999e-10
3.498347931846254e-11
1.556770525419753e-11
1.0961867041639217e-11
1.1087115879690822e-12
6.161872812065389e-12
2.4065795443309865e-12
1.5014752716777006e-09
1.5117457305320817e-09
1.956359991892391e-09
2.2657472980822938e-09
2.1203306627473968e-09
1.3439307610367588e-09
9.767132219121187e-10
2110501,0.042,0

2110502,0.127,0

2110503,0.064,0

2110504,0.061,0

2110505,0.07,0

2110506,0.082,0

2110507,0.099,0

2110508,0.064,0

2110509,0.095,0

2110510,0.092,0

2110512,0.067,0

2110514,0.079,0

2110515,0.0